In [1]:
import numpy as np
from numba import njit
import pickle
from sklearn.ensemble import BaggingClassifier
from sklearn.tree._classes import DecisionTreeClassifier
from joblib import Parallel, delayed
from scipy.stats import entropy, multivariate_normal

from hyppo.tools import multimodal_independence, indep_sim, rot_ksamp
from hyppo.tools import SIMULATIONS
from hyppo.ksample._utils import k_sample_transform
from tqdm import tqdm
from hyppo.tools import * 

import sys
import os
import multiprocessing as mp
from proglearn import UncertaintyForest
from joblib import Parallel, delayed
sys.path.append('C:\\Users\\siptest\\AppData\\Roaming\\Python\\Python36\\Scripts')
%load_ext autoreload
%autoreload 2

In [2]:
""""class TestStat:
    def __init__(self, uf, oob_func, n_jobs=1):
        self.uf = uf
        self.fit_ = False
        
    def statistic(self, X, y):
        y = y.ravel()
        if not self.fit_:
            self.uf = self.uf.fit(X, y)
            if hasattr(self, "leaf_indices_"):
                del self.leaf_indices_
            self.fit_ = True
        if not hasattr(self, "leaf_indices_"):
            self.leaf_indices_ = []
            for tree, voter_indices in zip(self.uf.transformers_, self.uf.voter_indices_):
                self.leaf_indices_.append(tree.apply(X[voter_indices]))

        _, counts = np.unique(y, return_counts=True)
        H_Y = entropy(counts, base=np.exp(1))
        H_YX = self._refit_apply_oob(X, y)
        return H_Y - H_YX

    def _refit_apply_oob(self, X, y):
        posteriors = np.zeros((X.shape[0], self.uf.n_classes_), dtype=np.float64)
        # posteriors = np.zeros((X.shape[0], self.uf.n_estimators, self.uf.n_classes_), dtype=np.float64)

        for tree, leaf_indices, voter_indices in zip(self.uf.transformers_, self.leaf_indices_, self.uf.voter_indices_):
        # for tree_idx, (leaf_indices, voter_indices) in enumerate(zip(self.leaf_indices_, self.uf.voter_indices_)):
            tree.tree_.value[:, :, :] = 0
            # print(tree.tree_.value.shape)
            # print(leaf_indices)

            for leaf_id, yval in zip(leaf_indices, y[voter_indices]):
                tree.tree_.value[leaf_id][0, yval] += 1
                # posteriors[leaf_id,]

            proba = tree.tree_.value[leaf_indices][:, 0, :]
            normalizer = proba.sum(axis=1)[:, np.newaxis]
            normalizer[normalizer == 0.0] = 1.0 # solves divide by 0 error
            proba /= normalizer
            posteriors[voter_indices] += proba

        # Normalize to unit length, due to prior weighting
        zero_mask = posteriors.sum(1) == 0
        posteriors[~zero_mask] /= posteriors[~zero_mask].sum(1, keepdims=True)
        posteriors[zero_mask] = self.uf.empirical_prior_

        return np.mean(entropy(posteriors, base=np.exp(1), axis=1))"""

class UFTest:
    def __init__(self, uf):
        self.uf = uf
        self.fit_ = False
        
    def statistic(self, X, y):
        y = y.ravel()
        if not self.fit_:
            self.uf = self.uf.fit(X, y)
            if hasattr(self, "honest_leaf_indices_"):
                assert hasattr(self, "transformer_leaf_indices_")
                del self.honest_leaf_indices_
                del self.transformer_leaf_indices_
            self.fit_ = True
        if not hasattr(self, "honest_leaf_indices_"):
            self.honest_leaf_indices_ = []
            self.transformer_leaf_indices_ = []
            for tree, voter_indices, transformer_indices in zip(self.uf.transformers_, self.uf.voter_indices_, self.uf.transformer_indices_):
                self.honest_leaf_indices_.append(tree.apply(X[voter_indices]))
                self.transformer_leaf_indices_.append(tree.apply(X[transformer_indices]))

        _, counts = np.unique(y, return_counts=True)
        H_Y = entropy(counts, base=np.exp(1))
        H_YX = self._refit_apply_oob(X, y)
        return max(H_Y - H_YX, 0)

    def _refit_apply_oob(self, X, y):
        posteriors = np.zeros((X.shape[0], self.uf.n_classes_), dtype=np.float64)
        # posteriors = np.zeros((X.shape[0], self.uf.n_estimators, self.uf.n_classes_), dtype=np.float64)

        for tree, honest_leaf_indices, voter_indices, tranformer_leaf_indices, transformer_indices in zip(
            self.uf.transformers_, self.honest_leaf_indices_, self.uf.voter_indices_, self.transformer_leaf_indices_, self.uf.transformer_indices_):
        # for tree_idx, (honest_leaf_indices, voter_indices) in enumerate(zip(self.honest_leaf_indices_, self.uf.voter_indices_)):
            tree.tree_.value[:, :, :] = 0

            for leaf_id, yval in zip(honest_leaf_indices, y[voter_indices]):
                tree.tree_.value[leaf_id][0, yval] += 1

            leaf_indices = np.hstack((honest_leaf_indices, tranformer_leaf_indices))
            sample_indices = np.hstack((voter_indices, transformer_indices))

            proba = tree.tree_.value[leaf_indices][:, 0, :]
            normalizer = proba.sum(axis=1)[:, np.newaxis]
            normalizer[normalizer == 0.0] = 1.0 # solves divide by 0 error
            proba /= normalizer
            posteriors[sample_indices] += proba

        # Normalize to unit length, due to prior weighting
        zero_mask = posteriors.sum(1) == 0
        posteriors[~zero_mask] /= posteriors[~zero_mask].sum(1, keepdims=True)
        posteriors[zero_mask] = self.uf.empirical_prior_

        return np.mean(entropy(posteriors, base=np.exp(1), axis=1))



In [3]:
def find_dim(sim):
    """Find dimension maximum for the simulation."""
    if sim not in SIMULATIONS.keys():
        raise ValueError("Invalid simulation")

    if sim in ["joint_normal", "sin_four_pi", "sin_sixteen_pi", "multiplicative_noise"]:
        dim = 10
    elif sim in ["multimodal_independence", "uncorrelated_bernoulli", "logarithmic"]:
        dim = 100
    elif sim in ["linear", "exponential", "cubic"]:
        dim = 1000
    elif sim in ["square", "diamond"]:
        dim = 40
    else:
        dim = 20

    return dim

def find_dim_range(dim):
    """Create list of dimension range to calculate power for."""
    if dim < 20:
        lim = 10
    else:
        lim = 20

    dim_range = list(range(int(dim / lim), dim + 1, int(dim / lim)))
    if int(dim / lim) != 1:
        dim_range.insert(0, 1)
    return dim_range

In [4]:
""""def _perm_stat(calc_stat, x, y):
    permy = np.random.permutation(y)
    perm_stat = calc_stat.statistic(x, permy)

    return perm_stat


def perm_test(calc_stat, X, y, reps=1000, workers=1):
    
    #Calculate the p-value via permutation
    
    # calculate observed test statistic
    stat = calc_stat.statistic(X, y)
    
    null_dist = np.asarray([_perm_stat(calc_stat, X, y) for rep in range(reps)])
    pvalue = ((null_dist >= stat).sum() + 1) / (reps + 1)


    return stat, pvalue"""

def _perm_stat(calc_stat, x, y):
    permy = np.random.permutation(y)
    perm_stat = calc_stat.statistic(x, permy)

    return perm_stat


def perm_test(calc_stat, X, y, reps=1000):
    """
    Calculate the p-value via permutation
    """
    # calculate observed test statistic
    if hasattr(calc_stat, "fit_"):
        calc_stat.fit_ = False
    stat = calc_stat.statistic(X, y)
    
    null_dist = np.asarray([_perm_stat(calc_stat, X, y) for rep in range(reps)])
    pvalue = ((null_dist >= stat).sum() + 1) / (reps + 1)

    return stat, pvalue, null_dist

In [7]:
POWER_REPS = 100

def estimate_power(sim, test):
    
    est_power = []
    dim_range = find_dim_range(find_dim(sim))
    for i in tqdm(dim_range):
        print("dimension " + str(i))
        print(sim)
    
        pvalues = []
        for _ in tqdm(range(POWER_REPS)): 
            x, y = rot_ksamp(sim, n=100, p=i, noise=True)
            X, y = k_sample_transform([x, y])
            y = y.astype(int)
            y = y.ravel()
            print(X.shape)
            print(y.shape)
            uf = UncertaintyForest(
                n_estimators=100,
                tree_construction_proportion=0.5,
                kappa=np.inf,
                honest_prior="ignore",
                max_features=1.0,
                n_jobs=-2,
            )
            uf = uf.fit(X, y)
            print('Fitted')

            reps = 1000
            test = UFTest(uf)
            stat, pvalue, null_dist = perm_test(test, X, y, reps=reps)
            print("Test Statistic: ", stat)
            print("p-value: ", pvalue)

            """"
            ce = lambda x: entropy(x, axis=1, base=np.exp(1))
            obs_stat = UFTest(uf, ce)

            reps = 1000
            stat, pvalue = perm_test(obs_stat, X, y, reps=reps, workers=-2)
            print(pvalue)
            """
            
            pvalues.append(pvalue)
            with open("./type_vs_attributes_test.csv", "w") as f:
                f.write(f"Test Statistic,p-value\n{stat},{pvalue}")
        num = 0
        for pval in pvalues: 
            if pval <= 0.05: 
                num += 1
        empirical_power = (1+num) / (1+POWER_REPS)
        est_power.append(empirical_power)
        
    print(est_power)
        
    
    print(sim + " done")
    np.savetxt('C:/Users/siptest/Desktop/NDD/{}_{}_vs_Dimension_HonestSampling_5_23.csv'.format(sim, "UF"),
            est_power, delimiter=',')
    
    return est_power

In [ ]:
for sim_name in tqdm(SIMULATIONS.keys()): 
    if sim_name != 'linear' and sim_name != 'exponential' and sim_name!= 'cubic' and sim_name != 'joint_normal' and sim_name != 'step' and sim_name != 'quadratic':
        power = estimate_power(sim_name, "UF")

In [8]:
estimate_power("circle", "UF")

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

dimension 1
circle
(200, 2)
(200,)
Fitted



  1%|▊                                                                                 | 1/100 [00:31<51:43, 31.35s/it]

Test Statistic:  0.2044395607309899
p-value:  0.000999000999000999
(200, 2)
(200,)
Fitted



  2%|█▋                                                                                | 2/100 [01:00<50:17, 30.79s/it]

...


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [49:56<00:32, 32.81s/it]

Test Statistic:  0.0034880344032354182
p-value:  1.0
(200, 40)
(200,)
Fitted



100%|█████████████████████████████████████████████████████████████████████████████| 20/20 [17:24:25<00:00, 3133.25s/it]

Test Statistic:  0.012324810661255126
p-value:  0.06993006993006994
[1.0, 0.09900990099009901, 0.19801980198019803, 0.15841584158415842, 0.21782178217821782, 0.18811881188118812, 0.1485148514851485, 0.2376237623762376, 0.21782178217821782, 0.2376237623762376, 0.18811881188118812, 0.1485148514851485, 0.13861386138613863, 0.16831683168316833, 0.1485148514851485, 0.15841584158415842, 0.15841584158415842, 0.1188118811881188, 0.1485148514851485, 0.13861386138613863]
circle done


[1.0,
 0.09900990099009901,
 0.19801980198019803,
 0.15841584158415842,
 0.21782178217821782,
 0.18811881188118812,
 0.1485148514851485,
 0.2376237623762376,
 0.21782178217821782,
 0.2376237623762376,
 0.18811881188118812,
 0.1485148514851485,
 0.13861386138613863,
 0.16831683168316833,
 0.1485148514851485,
 0.15841584158415842,
 0.15841584158415842,
 0.1188118811881188,
 0.1485148514851485,
 0.13861386138613863]

In [ ]:
sim = 'linear'

est_power = np.genfromtxt('C:/Users/siptest/Desktop/NDD/{}_{}_vs_Dimension_HonestSampling_5_16.csv'.format(sim, "UF"),
delimiter=",")

dim_range = find_dim_range(find_dim(sim))

col.plot(dim_range, est_power, label=INDEP_TESTS[test].__name__, lw=2)
col.set_xticks([])
if i == 3:
col.set_xticks([dim_range[0], dim_range[-1]])
col.set_ylim(-0.05, 1.05)
col.set_yticks([])
if j == 0:
col.set_yticks([0, 1])
col.set_title(SIM_TITLES[count])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[1:], n_colors=9)

from hyppo.tools import INDEP_TESTS

SIM_TITLES = [
    "Linear",
    "Exponential",
    "Cubic",
    "Joint Normal",
    "Step",
    "Quadratic",
    "W-Shaped",
    "Spiral",
    "Bernoulli",
    "Logarithmic",
    "Fourth Root",
    "Sine 4\u03C0",
    "Sine 16\u03C0",
    "Square",
    "Two Parabolas",
    "Circle",
    "Ellipse",
    "Diamond",
    "Noise",
    "Independence",
]

def plot_power_all():
    fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(25, 20))
    plt.suptitle(
        "Multivariate Independence Testing (Increasing Dimension)",
        y=0.93,
        va="baseline",
    )
    
    for i, row in enumerate(ax):
        for j, col in enumerate(row):
            count = 5 * i + j
            sim = list(SIMULATIONS.keys())[count]

            for test in INDEP_TESTS.keys():
                if test == 'kmerf' or test == 'maxmargin':
                    continue
                
                est_power = np.genfromtxt(
                    "C:/Users/siptest/hyppo/benchmarks/vs_dimension/{}_{}.csv".format(sim, test),
                    delimiter=",")
                
                dim_range = find_dim_range(find_dim(sim))

                col.plot(dim_range, est_power, label=INDEP_TESTS[test].__name__, lw=2)
                col.set_xticks([])
                if i == 3:
                    col.set_xticks([dim_range[0], dim_range[-1]])
                col.set_ylim(-0.05, 1.05)
                col.set_yticks([])
                if j == 0:
                    col.set_yticks([0, 1])
                col.set_title(SIM_TITLES[count])
                
            if sim == "linear":
                est_power = np.genfromtxt('C:/Users/siptest/Desktop/NDD/{}_{}_vs_Dimension_HonestSampling_5_2.csv'.format(sim, "UF"),
                                  delimiter=',')
            else:
                est_power = np.genfromtxt('C:/Users/siptest/Desktop/NDD/{}_{}_vs_Dimension_HonestSampling_5_2.csv'.format(sim, "UF"),
                                      delimiter=',')
            dim_range = find_dim_range(find_dim(sim))

            col.plot(dim_range, est_power, label="UF", lw=2)
            col.set_xticks([])
            if i == 3:
                col.set_xticks([dim_range[0], dim_range[-1]])
            col.set_ylim(-0.05, 1.05)
            col.set_yticks([])
            if j == 0:
                col.set_yticks([0, 1])
            col.set_title(SIM_TITLES[count])
            
    fig.text(0.5, 0.05, "Dimension", ha="center")
    fig.text(
        0.07,
        0.5,
        "Statistical Power",
        va="center",
        rotation="vertical",
    )
    leg = plt.legend(
        bbox_to_anchor=(0.5, 0.05),
        bbox_transform=plt.gcf().transFigure,
        ncol=len(INDEP_TESTS.keys()),
        loc="upper center",
    )
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.subplots_adjust(hspace=0.50)
    plt.savefig('C:/Users/siptest/Desktop/NDD/indep_power_dimension_allSim_HonestUF_5_23.png', box_inches='tight')
    #, box_inches='tight'


In [ ]:
plot_power_all()